# 🚀 GEAK-Agent Tutorial
### Generating Optimized GPU Kernels with LLM Agents

---

This notebook demonstrates **GEAK-Agent** - a multi-agent LLM framework that automatically generates and optimizes [Triton](https://github.com/openai/triton) GPU kernels.

| Step | What Happens |
|:----:|-------------|
| 1️⃣ | **Setup** - Configure API keys |
| 2️⃣ | **Initialize** - Load kernels and create agent |
| 3️⃣ | **Run** - Generate and optimize kernels |
| 4️⃣ | **Results** - View generated code and metrics |

---
## 1️⃣ Setup

In [1]:
# Initialize environment
import os, sys

# Auto-detect paths
TUTORIAL_DIR = os.getcwd() if os.getcwd().endswith('tutorial') else os.path.join(os.getcwd(), 'tutorial')
GEAK_DIR = os.path.dirname(TUTORIAL_DIR)
SRC_DIR = os.path.join(GEAK_DIR, 'src')

# Add src first (so src/utils doesn't get shadowed)
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)
if TUTORIAL_DIR not in sys.path:
    sys.path.insert(0, TUTORIAL_DIR)

# Import tutorial utilities
from tutorial_utils import setup_environment, print_header
TUTORIAL_DIR, SRC_DIR, CORPUS_PATH, TutorialDataloader = setup_environment()
print_header('✓ GEAK-Agent Tutorial Ready')

╔════════════════════════════════════════════════════════════╗
║ ✓ GEAK-Agent Tutorial Ready                                ║
╚════════════════════════════════════════════════════════════╝


In [3]:
# ═══════════════════════════════════════════════════════════════
#                       🔑 API CONFIGURATION
# ═══════════════════════════════════════════════════════════════

USE_AMD_API = True                              # Set False for OpenAI
AMD_API_KEY = '471c248fdb454e8b96173c8d25b03593'                # Your AMD API key
OPENAI_API_KEY = 'YOUR_OPENAI_KEY'              # Your OpenAI key

# ───────────────────────────────────────────────────────────────
API_KEY = AMD_API_KEY if USE_AMD_API else OPENAI_API_KEY
MODEL_ID = 'GPT4o' if USE_AMD_API else 'gpt-4o'
print(f'✓ API: {"AMD" if USE_AMD_API else "OpenAI"} | Model: {MODEL_ID}')

✓ API: AMD | Model: GPT4o


---
## 2️⃣ Initialize Dataset & Agent

In [4]:
from models.OpenAI import OpenAIModel, StandardOpenAIModel
from agents.GaAgent import GaAgent

# Load kernels
dataset = TutorialDataloader(
    kernel_names=['add_example.py', 'sin_computation.py'],
    corpus_path=CORPUS_PATH
)

# Initialize model & agent
model = OpenAIModel(api_key=API_KEY, model_id=MODEL_ID) if USE_AMD_API \
    else StandardOpenAIModel(api_key=API_KEY, model_id=MODEL_ID)

agent = GaAgent(model=model, dataset=dataset, corpus_path=CORPUS_PATH, descendant_num=1)

print(f'✓ Loaded {len(dataset)} kernels | Agent ready with {len(agent.memories)} tasks')

✓ Loaded 2 kernels | Agent ready with 2 tasks


In [5]:
# Preview kernel instructions (optional)
from tutorial_utils import display_kernel_info
display_kernel_info(dataset.problem_states)


┌──────────────────────────────────────────────────────────────────────┐
│ 📄 add_example.py                                                   │
├──────────────────────────────────────────────────────────────────────┤
│ You are a expert in writing Triton operators for efficient GPU progr │
│ amming. Use triton language write a kernel and wrapper according fol │
│ lowing instruction.             The Triton code provides an element- │
│ wise addition kernel named `add_kernel` and a corresponding wrapper  │
│ function `add_wrapper`. The `add_kernel` is a decorated function wit │
│ h `@triton.jit`, enabling Just-In-Time compilation for optim         │
│ ...[truncated]                                                       │
├──────────────────────────────────────────────────────────────────────┤
│ ✓ Has reference code: True                                          │
│ ✓ Has test code: True                                               │
└─────────────────────────────────────────────────────

---
## 3️⃣ Run GEAK-Agent

Each iteration: **Generate** → **Test** → **Benchmark** → **Reflect**

> ⏱️ Takes a few minutes per iteration

In [6]:
from tutorial_utils import print_config

OUTPUT_DIR = os.path.join(TUTORIAL_DIR, 'outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ═══════════════════════════════════════════════════════════════
#                       ⚙️ CONFIGURATION
# ═══════════════════════════════════════════════════════════════
CONFIG = {
    'iteration_num': 3,       # Optimization iterations
    'temperature': 1.0,       # LLM creativity (0.0-2.0)
    'descendant_num': 1,      # Candidates per iteration
    'ancestor_num': 3,        # Reference solutions
    'gpu_id': 0,              # GPU device
    'target_gpu': 'MI325X',   # Target AMD GPU
}
print_config(CONFIG, '🚀 Running GEAK-Agent')


════════════════════════════════════════════════════════════
 🚀 Running GEAK-Agent
════════════════════════════════════════════════════════════
  • iteration_num: 3
  • temperature: 1.0
  • descendant_num: 1
  • ancestor_num: 3
  • gpu_id: 0
  • target_gpu: MI325X
════════════════════════════════════════════════════════════


In [ ]:
# Run the agent
agent.run(
    output_path=os.path.join(OUTPUT_DIR, 'tutorial_results.jsonl'),
    multi_thread=False,
    **CONFIG,
    start_iter=0, descendant_debug=0, profiling=False, start_idx=0
)
print('\n✅ Complete!')

---
## 4️⃣ View Results

In [9]:
from tutorial_utils import load_results, display_results_summary, display_generated_code

results, iteration = load_results(OUTPUT_DIR)
display_results_summary(results, iteration)


╔══════════════════════════════════════════════════════════════════════╗
║ 📊 GEAK-Agent Results Summary                                         ║
║ Iteration: 3                                                         ║
╠══════════════════════════════════════════════════════════════════════╣
║ Kernel              │ Call │ Exec │ Perf │ Speedup                   ║
╠──────────────────────────────────────────────────────────────────────╣
║ add_example.py     │  ✓   │  ✓   │  ✓   │ 1.0049x                    ║
║ sin_computation.py │  ✓   │  ✓   │  ✓   │ 1.0085x                    ║
╚══════════════════════════════════════════════════════════════════════╝

 Legend: ✓ = Passed, ✗ = Failed
 Speedup: reference_time / generated_time (>1.0 = faster)


In [10]:
# View generated code
display_generated_code(results)


┌──────────────────────────────────────────────────────────────────────┐
│ 🔧 add_example.py - ✓ Correct                                         │
│    Speedup: 1.0049x                                                  │
└──────────────────────────────────────────────────────────────────────┘
import torch
import triton
import triton.language as tl

@triton.jit
def add_kernel(
    in_ptr0,
    in_ptr1,
    out_ptr,
    n_elements,
    BLOCK_SIZE: tl.constexpr,
):
    pid = tl.program_id(0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)

    # Mask for out-of-bounds
    mask = offsets < n_elements

    # Use shared memory to cache input values for reuse
    shared_val0 = tl.load(in_ptr0 + offsets, mask=mask, other=0.0)
    shared_val1 = tl.load(in_ptr1 + offsets, mask=mask, other=0.0)

    # Element-wise addition
    result = shared_val0 + shared_val1

    # Store results in output tensor with masking
    tl.store(out_ptr + offsets, result, mask=ma

---
## 📚 Reference

| Metric | Meaning |
|--------|--------|
| **pass_call** | Code runs without errors |
| **pass_exe** | Output matches reference |
| **speedup** | `ref_time/gen_time` (>1 = faster) |

**Tips:** More iterations → better results • Higher temperature → more diversity

---
🎉 **Done!** See [GEAK-Agent](https://github.com/AMD-AGI/GEAK-agent) for more.